## Dependencies

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

plt.rcParams["animation.html"] = "jshtml"

## 1. Independent Particles in Motion
This code simulates the motion of particles with randomized positions and velocities in a 2D space, without considering gravity.

In [ ]:
N_PARTICLES = 20
G = 6.674e-11

positions = np.random.uniform(-100,100,(N_PARTICLES,2))
velocities = np.random.uniform(-10,10,(N_PARTICLES,2))
p_list = []
dt = 0.2 # time step between each frame

def animate(t):
    plt.cla()
    plt.grid()
    
    
    #plt.xlim(0,300)
    #plt.ylim(0,150)
    
    global positions
    
    positions = positions + velocities*dt
    ax.scatter(0,0,c='r',marker='+')
    ax.scatter(positions[:,0],positions[:,1],s=30,alpha=0.5)
    
    x_min, x_max = ax.get_xlim()
    y_min, y_max = ax.get_ylim()
    ax.set_xticks(np.arange(x_min-50,x_max+50,(x_max-x_min)/10))
    ax.set_yticks(np.arange(y_min-50,y_max+50,(y_max-y_min)/10))
    ax.set_title('Particles in Motion')
    
    if len(p_list) != 0:
        # contrail
        for particle_i in range(N_PARTICLES):
            line = []
            for history_j in range(len(p_list)):
                line.append(p_list[history_j][particle_i])

            line = np.array(line)
            
            alpha_vals = 0.2 if len(p_list) == 1 else np.linspace(0.1,0.2,len(p_list))

            ax.plot(line[:,0],line[:,1],'--',alpha=0.2)
            
    p_list.append(positions)
    
# plot the figure
fig, ax = plt.subplots(dpi=100,figsize=(8,8))
ax.scatter(0,0,c='r',marker='+')
ax.scatter(positions[:,0],positions[:,1],s=30,alpha=0.5)

# save the animation
ani1 = animation.FuncAnimation(fig, animate, frames=150,interval=30)

![Alt Text](animations/ani1.gif)

## 2. Interacting Bodies: Orbital Simulation using Newton's Law
Using Newton's law, let's simulate a world with 5 celestial bodies. We'll compute the gravitational interaction between all the bodies in the system by taking into account their mass, diameter, velocity, and distance from one another.  

### Law of Gravitation
* F = G * (m1 * m2) / r^2, where F is the gravitational force between two masses m1 and m2 separated by a distance r, and G is the gravitational constant (6.674 x 10^-11)

### Notes
* Initial velocities are set to zero
* Initial positions are normally randomized between (-20,20) on the x and y plane
* Mass is uniformly randomized between (50,100)
* The gravitational constant is set to 1e-5 for visual scaling

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# constants
G = 0.00001 # gravitational constant

# particle properties
N_BODIES = 5
mass = np.random.uniform(50, 100, (N_BODIES,1)) # masses of particles
positions = np.random.normal(-20,20,(N_BODIES,2))
velocities = np.zeros((N_BODIES,2))
forces = np.zeros((N_BODIES, 2))  # initialize forces to zero

# setup history tracking for each particles position
p_history = {}
for n in range(N_BODIES):
    p_history[n] = []

# calculate acceleration due to gravity for each particle
def calc_acceleration():
    acceleration = np.zeros((N_BODIES, 2))
    for i in range(N_BODIES):
        for j in range(N_BODIES):
            if i != j:
                r = positions[j] - positions[i]
                dist = np.linalg.norm(r)
                f = G * mass[i] * mass[j] / dist
                forces[i] += f * r / dist
        acceleration[i] = forces[i] / mass[i]
    return acceleration

# update particle positions and velocities
def update(dt):
    global positions, velocities, forces
    acceleration = calc_acceleration()
    positions += velocities * dt + 0.5 * acceleration * dt ** 2
    velocities += 0.5 * (forces / mass) * dt + 0.5 * acceleration * dt
    forces = np.zeros((N_BODIES, 2))
    
    for n in range(N_BODIES):
        p_history[n].append((positions[n,0],positions[n,1]))

trace_max = 30
def animate(frame):
    update(30) # update interval per frame = 20 seconds
    #scat.set_offsets(positions)
    plt.cla()
    plt.grid()
    ax.scatter(positions[:, 0], positions[:, 1], s=mass, alpha=0.8)
    frame_count = len(p_history[0])
    ax.set_title(f'Orbital Simulation using Newtons Law (t: {frame_count})')
    
    for n in range(N_BODIES):
        #line = np.array(p_history[n])
        line = np.array(p_history[n][-np.min([trace_max,frame_count]):])
        ax.plot(line[:,0],line[:,1],'--',alpha=0.2)
    
    return scat,

# plot the figure
fig, ax = plt.subplots(dpi=100,figsize=(14,8))
plt.grid()
ax.set_title('Orbital Simulation using Newtons Law')
scat = ax.scatter(positions[:, 0], positions[:, 1], s=mass)

# save the animation
ani2 = animation.FuncAnimation(fig, animate, frames=400, interval=30, blit=False)

![Alt Text](animations/ani2a.gif)

![Alt Text](animations/ani2b.gif)

## 3. Milky Way: Orbital Model of the Solar System
Now that we've developed a simple orbital model using Newton's Gravitational Law, let's apply the same concept for simulating the Milky Way. 

### Computation
This model uses Verlet integration to update the position, velocity, and acceleration of each celestial body. First, the function compute_acceleration calculates the acceleration at each body's current position, which is then used to update the positions using Verlet integration. Then, the function recalculates the acceleration at the new positions, and the velocities are updated using Verlet's integration as well. The resulting positions, velocities, and accelerations are used to plot the trajectories of the bodies in the Milky Way for each time-step of 1 day (86,400 seconds).

### Notes
* Delta-T is one earth year
* For scaling, Uranus and Neptune were omitted (allows us to see the orbits of Earth and Mars more clearly)
* For visualization, the animated size of the Sun was scaled down by 95%

In [ ]:
# constants
G = 6.67430e-11  # Gravitational constant, N m^2/kg^2
AU = 1.496e11    # Astronomical unit in meters
dt = 60*60*24 # seconds per day

# Milky Way dict containing information about the bodies (diameter: km, mass: kg, position: km, velocity: km/s)
milky_way = {
    "Sun": {
        "diameter": 1392700,
        "mass": 1.989e30,
        "position": np.array([0, 0]),
        "velocity": np.array([0, 0])
    },
    "Mercury": {
        "diameter": 4879,
        "mass": 3.285e23,
        "position": np.array([57909050, 0]),
        "velocity": np.array([0, 47.362])
    },
    "Venus": {
        "diameter": 12104,
        "mass": 4.867e24,
        "position": np.array([108208000, 0]),
        "velocity": np.array([0, 35.020])
    },
    "Earth": {
        "diameter": 12756,
        "mass": 5.972e24,
        "position": np.array([147098291, 0]),
        "velocity": np.array([0, 29.783])
    },
    "Mars": {
        "diameter": 6792,
        "mass": 6.39e23,
        "position": np.array([206655215, 0]),
        "velocity": np.array([0, 24.140])
    },
    "Jupiter": {
        "diameter": 139822,
        "mass": 1.898e27,
        "position": np.array([740679835, 0]),
        "velocity": np.array([0, 13.070])
    },
    "Saturn": {
        "diameter": 116460,
        "mass": 5.683e26,
        "position": np.array([1353572956, 0]),
        "velocity": np.array([0, 9.690])
    },
    # "Uranus": {
    #     "diameter": 50724,
    #     "mass": 8.681e25,
    #     "position": np.array([2748390901, 0]),
    #     "velocity": np.array([0, 6.810])
    # },
    # "Neptune": {
    #     "diameter": 49244,
    #     "mass": 1.024e26,
    #     "position": np.array([4452940833, 0]),
    #     "velocity": np.array([0, 5.430])
    # }
}

# initial positions and velocities of the celestial bodies
positions = np.array([body["position"] for body in milky_way.values()], dtype=np.float64) * 1000 # convert km to meters
velocities = np.array([body["velocity"] for body in milky_way.values()], dtype=np.float64) * 1000 # convert km/s to meters/s
masses = np.array([body["mass"] for body in milky_way.values()], dtype=np.float64)
diameters = np.array([body["diameter"] for body in milky_way.values()], dtype=np.float64)
diameters[0] *= 0.05 # the sun scaled down by 95%
diameters = (diameters / diameters.max()) * 200 # scaled-down sizing for plotting

# initialize p_history – we'll use this dict to track the orbits of all the bodies
p_history = {}
for k in milky_way.keys():
    p_history[k.lower()] = []

def compute_acceleration(positions, masses):
    acceleration = np.zeros_like(positions)
    for i in range(len(masses)):
        # get mass and position for body i
        m1 = masses[i]
        x1,y1 = positions[i]
        
        for j in range(len(masses)):
            if i == j:
                continue
            
            # get mass and position for body j
            m2 = masses[j]
            x2,y2 = positions[j]
            
            # distance between two points
            dist = np.sqrt((y2-y1)**2+(x2-x1)**2)
            
            # angle between two points
            theta = np.arctan2(y2-y1, x2-x1)
            
            # force calculations
            F = G*(m1*m2)/dist**2 # force between two points
            Fx = F * np.cos(theta) # force in x axis
            Fy = F * np.sin(theta) # force in y axis
            
            # append forces to objects current acceleration
            acceleration[i,0] += Fx/m1
            acceleration[i,1] += Fy/m1
            
    return acceleration

def update(positions, velocities):
    acceleration = compute_acceleration(positions, masses) # compute acceleration at current positions
    positions += (velocities * dt) + (0.5 * acceleration * dt**2) # use verlet integration to update mew positions
    new_acceleration = compute_acceleration(positions, masses) # compute acceleration at new positions
    velocities += 0.5 * (acceleration + new_acceleration) * dt # use verlet integration to update velocities
    return positions,velocities

def animate(frame):
    # call variables globally and update position, velocity, acceleration per frame
    global positions, velocities, masses
    positions,velocities = update(positions,velocities)
    
    # convert positions to AU for scaling
    positions_au = positions / AU

    # clear plot and set grid
    plt.cla()
    plt.grid()
    
    # set axis limits
    ax.set_xlim(-2, 10)
    ax.set_ylim(-2, 3.5)

    # plot celestial bodies on scatter plot
    ax.scatter(positions_au[:, 0], positions_au[:, 1], s=diameters)
    
    # set title and xy labels
    ax.set_title(f"Milky Way Sim (Day {frame:00.0f})")
    ax.set_xlabel('X (AU)')
    ax.set_ylabel('Y (AU)')
    
    # plot the name and trajectories of each celestial body
    for z in range(len(positions)):
        planet = list(milky_way.keys())[z].lower()
        p_history[planet].append((positions_au[z,0],positions_au[z,1]))
        line = np.array(p_history[planet])
        ax.plot(line[:,0],line[:,1],'--',alpha=0.2) # plot trajectory
        ax.text(positions_au[z, 0], positions_au[z, 1],planet,fontsize=6) # plot name
    

# plot the figure
fig, ax = plt.subplots(dpi=100,figsize=(14,8))
ax.set_xlabel('X (AU)')
ax.set_ylabel('Y (AU)')
ax.set_title('Milky Way Sim')

# save the animation
ani3 = animation.FuncAnimation(fig, animate, frames=365, interval=20, blit=False)

![Alt Text](animations/ani3.gif)